In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
raddar_chest_xrays_indiana_university_path = kagglehub.dataset_download('raddar/chest-xrays-indiana-university')

print('Data source import complete.')


100%|██████████| 13.2G/13.2G [10:42<00:00, 22.0MB/s]

Extracting files...


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

till now best model

In [3]:
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageDraw
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda


In [5]:
###########################
### 1. Data Preparation ###
###########################
# Add this custom collate function before creating DataLoaders
def custom_collate_fn(batch):
    """Handle mixed data types in batch"""
    elem = batch[0]
    collated = {}

    for key in elem:
        if isinstance(elem[key], torch.Tensor):
            # Stack tensors
            collated[key] = torch.stack([item[key] for item in batch])
        elif isinstance(elem[key], (int, float)):
            # Convert scalars to tensor
            collated[key] = torch.tensor([item[key] for item in batch])
        else:
            # Keep as list (strings, dicts, etc)
            collated[key] = [item[key] for item in batch]

    return collated

class RadiologyDataset(Dataset):
    def __init__(self, root_dir, report_csv, projection_csv, transform=None, max_samples=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_dir = os.path.join(root_dir, "images", "images_normalized")

        # Load and merge metadata
        reports_df = pd.read_csv(os.path.join(root_dir, report_csv))
        projections_df = pd.read_csv(os.path.join(root_dir, projection_csv))

        # Merge on UID
        self.metadata = pd.merge(
            reports_df,
            projections_df,
            on="uid",
            how="inner"
        )

        # Filter frontal X-rays only
        self.metadata = self.metadata[self.metadata['projection'].str.lower() == 'frontal']
        # Drop rows where image file doesn't exist
        self.metadata['img_path'] = self.metadata['filename'].apply(lambda f: os.path.join(self.image_dir, f))
        self.metadata = self.metadata[self.metadata['img_path'].apply(os.path.exists)]

        # THEN do sampling
        if max_samples:
            self.metadata = self.metadata.sample(min(max_samples, len(self.metadata)), random_state=42)
        # Define common abnormalities
        self.abnormalities = [
            'cardiomegaly', 'edema', 'consolidation', 'pneumonia', 'atelectasis',
            'pneumothorax', 'pleural effusion', 'fracture', 'nodule', 'emphysema',
            'infiltration', 'mass', 'fibrosis', 'calcification'
        ]


        # Create full report text
        self.metadata['full_report'] = self.metadata.apply(
            lambda row: f"FINDINGS:\n{row['findings']}\n\nIMPRESSION:\n{row['impression']}",
            axis=1
        )
        # Precompute labels
        self.metadata['labels'] = self.metadata.apply(
            lambda row: [1 if abn in (str(row['findings']) + str(row['impression'])).lower() else 0
                         for abn in self.abnormalities],
            axis=1
        )



    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        img_path = row['img_path']

        try:
            image = Image.open(img_path).convert('L')  # Convert to grayscale
        except:
            # Use a blank image if loading fails
            image = Image.new('L', (256, 256), color=0)

        # Convert labels to tensor here
        labels = torch.tensor(row['labels'], dtype=torch.float)

        if self.transform:
            image = self.transform(image)

        return {
            'image': image,
            'uid': row['uid'],
            'full_report': row['full_report'],
            'labels': labels  # Only tensor now
        }

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # Grayscale normalization
])

# Create datasets
root_dir =raddar_chest_xrays_indiana_university_path  # Update with your path
dataset = RadiologyDataset(
    root_dir=root_dir,
    report_csv="indiana_reports.csv",
    projection_csv="indiana_projections.csv",
    transform=transform,
    max_samples=2000  # Reduce for Kaggle constraints
)

# Split dataset
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create data loaders with custom collate
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,
                          collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8,
                        collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8,
                         collate_fn=custom_collate_fn)

In [6]:
##############################################
### 2. Weakly-Supervised Detection Model ###
##############################################

class CAMModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Use a ResNet model modified for grayscale
        base_model = torchvision.models.resnet18(pretrained=True)

        # Modify first layer for grayscale (1 channel)
        base_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Feature extractor
        self.features = nn.Sequential(
            base_model.conv1,
            base_model.bn1,
            base_model.relu,
            base_model.maxpool,
            base_model.layer1,
            base_model.layer2,
            base_model.layer3,
            base_model.layer4
        )

        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)

        # Classifier

        self.classifier = nn.Linear(512, num_classes)

        # Hook for feature maps
        self.feature_maps = None
        self.features.register_forward_hook(self.get_activation())

    def get_activation(self):
        def hook(module, input, output):
            self.feature_maps = output
        return hook

    def forward(self, x):
        features = self.features(x)
        pooled = self.gap(features).flatten(1)
        return self.classifier(pooled)

    def get_cam(self, class_idx):
        weights = self.classifier.weight[class_idx]
        cam = torch.einsum('nchw,c->nhw', self.feature_maps, weights)
        return torch.relu(cam)

# Initialize model
model = CAMModel(len(dataset.abnormalities)).to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training function
def train_detector(model, train_loader, val_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        train_loss = 0.0

        for batch in train_loader:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Val Loss: {val_loss/len(val_loader):.4f}")

# Train the detector (reduce epochs for Kaggle)
train_detector(model, train_loader, val_loader, epochs=5)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 144MB/s]


Epoch 1/5 | Train Loss: 0.3525 | Val Loss: 0.2922
Epoch 2/5 | Train Loss: 0.2937 | Val Loss: 0.2980
Epoch 3/5 | Train Loss: 0.2888 | Val Loss: 0.2924
Epoch 4/5 | Train Loss: 0.2827 | Val Loss: 0.3086
Epoch 5/5 | Train Loss: 0.2785 | Val Loss: 0.2900


In [7]:
################################################
### 3. Generate Pseudo-Bounding Boxes (CAM) ###
################################################

def cam_to_bbox(cam, threshold=0.7, min_area=0.01):
    """Convert CAM to bounding boxes with filtering"""
    cam = cam.detach().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    mask = (cam > threshold).astype(np.uint8)

    from skimage.measure import label, regionprops
    labeled = label(mask)
    bboxes = []

    for region in regionprops(labeled):
        y1, x1, y2, x2 = region.bbox
        area = (x2 - x1) * (y2 - y1) / (cam.shape[1] * cam.shape[0])

        # Filter small regions
        if area < min_area:
            continue

        bboxes.append({
            'xmin': x1 / cam.shape[1],
            'ymin': y1 / cam.shape[0],
            'xmax': x2 / cam.shape[1],
            'ymax': y2 / cam.shape[0],
            'confidence': cam[region.slice].max(),
            'area': area
        })

    return bboxes

def generate_bboxes_for_dataset(model, dataset):
    """Add bounding boxes to dataset items"""
    model.eval()
    new_data = []

    for item in dataset:
        image = item['image'].unsqueeze(0).to(device)
        labels = item['labels'].cpu().numpy()

        with torch.no_grad():
            _ = model(image)

        bboxes = []
        for class_idx, label in enumerate(labels):
            if label == 1:  # Only for present abnormalities
                cam = model.get_cam(class_idx)
                class_bboxes = cam_to_bbox(cam[0])
                for bbox in class_bboxes:
                    bbox['class'] = dataset.dataset.abnormalities[class_idx]
                    bboxes.append(bbox)

        # If no findings, create full-image box
        if not bboxes and "no finding" in item['full_report'].lower():
            bboxes = [{
                'class': 'No finding',
                'xmin': 0,
                'ymin': 0,
                'xmax': 1,
                'ymax': 1,
                'confidence': 1.0,
                'area': 1.0
            }]

        # Sort by confidence and keep top 3
        bboxes = sorted(bboxes, key=lambda x: x['confidence'], reverse=True)[:3]
        new_item = item.copy()
        new_item['bboxes'] = bboxes
        new_data.append(new_item)

    return new_data

# Add bboxes to datasets
train_dataset_with_bbox = generate_bboxes_for_dataset(model, train_dataset)
val_dataset_with_bbox = generate_bboxes_for_dataset(model, val_dataset)
test_dataset_with_bbox = generate_bboxes_for_dataset(model, test_dataset)


In [9]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
####################################
### 4. Multi-View Image Creation ###
####################################

def create_multiview(item):
    """Generate global, saliency, and region views"""
    # Convert tensor to PIL image
    image_tensor = item['image']
    image_pil = transforms.ToPILImage()(image_tensor).convert("RGB")
    orig_size = image_pil.size

    # Global view
    global_view = image_pil.resize((224, 224))

    # Create heatmap overlay
    heatmap = Image.new('L', orig_size, 0)
    draw = ImageDraw.Draw(heatmap)

    for bbox in item['bboxes']:
        x1 = int(bbox['xmin'] * orig_size[0])
        y1 = int(bbox['ymin'] * orig_size[1])
        x2 = int(bbox['xmax'] * orig_size[0])
        y2 = int(bbox['ymax'] * orig_size[1])
        draw.rectangle([x1, y1, x2, y2], fill=int(255 * bbox['confidence']))

    # Blend with original image
    saliency_view = Image.blend(
        image_pil.convert("RGBA"),
        heatmap.convert("RGBA"),
        alpha=0.4
    ).convert("RGB").resize((224, 224))

    # Region crops
    region_views = []
    for bbox in item['bboxes']:
        x1 = int(bbox['xmin'] * orig_size[0])
        y1 = int(bbox['ymin'] * orig_size[1])
        x2 = int(bbox['xmax'] * orig_size[0])
        y2 = int(bbox['ymax'] * orig_size[1])
        w, h = x2 - x1, y2 - y1

        # Expand crop by 20% for context
        expand_x, expand_y = int(w * 0.2), int(h * 0.2)
        x1 = max(0, x1 - expand_x)
        y1 = max(0, y1 - expand_y)
        x2 = min(orig_size[0], x2 + expand_x)
        y2 = min(orig_size[1], y2 + expand_y)

        region = image_pil.crop((x1, y1, x2, y2)).resize((112, 112))
        region_views.append(region)

    # Pad to 3 regions
    while len(region_views) < 3:
        region_views.append(Image.new('RGB', (112, 112), (0, 0, 0)))

    # Convert to tensors
    transform_to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    return {
        'global_view': transform_to_tensor(global_view),
        'saliency_view': transform_to_tensor(saliency_view),
        'region1': transform_to_tensor(region_views[0]),
        'region2': transform_to_tensor(region_views[1]),
        'region3': transform_to_tensor(region_views[2]),
        'full_report': item['full_report'],
        'bboxes': item['bboxes']
    }

# Create multiview datasets
train_multiview = [create_multiview(item) for item in train_dataset_with_bbox]
val_multiview = [create_multiview(item) for item in val_dataset_with_bbox]
test_multiview = [create_multiview(item) for item in test_dataset_with_bbox]

# Initialize model with reduced parameters
VOCAB_SIZE = 5000

# Build vocabulary from training reports
word_counts = {}
for item in train_multiview:
    report = item['full_report'].lower()
    words = word_tokenize(report)
    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1

# Create vocabulary with special tokens
vocab = {
    '<PAD>': 0,
    '<SOS>': 1,
    '<EOS>': 2,
    '<UNK>': 3
}
# Add most common words
sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
for word, count in sorted_words:
    if len(vocab) < VOCAB_SIZE:  # Only keep top VOCAB_SIZE words
        vocab[word] = len(vocab)
    else:
        break

# Inverse vocabulary for decoding
inv_vocab = {idx: word for word, idx in vocab.items()}

# Text encoding function
def encode_caption(report, vocab, max_length=100):
    """Convert report text to sequence of token indices"""
    words = word_tokenize(report.lower())
    tokens = [vocab.get(word, vocab['<UNK>']) for word in words]
    tokens = [vocab['<SOS>']] + tokens + [vocab['<EOS>']]

    # Pad or truncate to max_length
    if len(tokens) < max_length:
        tokens += [vocab['<PAD>']] * (max_length - len(tokens))
    else:
        tokens = tokens[:max_length-1] + [vocab['<EOS>']]

    return torch.tensor(tokens, dtype=torch.long)

In [11]:
### 5. Multimodal Report Generation Model ###
##############################################

class MultimodalReportGenerator(nn.Module):
    def __init__(self, report_vocab_size, embedding_dim=256, hidden_dim=256):  # Reduced hidden_dim
        super().__init__()
        # Image encoder - shared weights
        base = torchvision.models.resnet18(pretrained=True)
        self.encoder = nn.Sequential(*list(base.children())[:-2])

        # Feature fusion
        self.feature_fusion = nn.Sequential(
            nn.Linear(512 * 5, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.embedding = nn.Embedding(report_vocab_size, embedding_dim)

        # Report decoder (LSTM)
        self.decoder = nn.LSTM(
            input_size=embedding_dim + 512,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_dim, report_vocab_size)

    def encode_view(self, view):
        """Encode a single view and apply adaptive pooling"""
        features = self.encoder(view)
        return nn.AdaptiveAvgPool2d(1)(features).squeeze(-1).squeeze(-1)

    def forward(self, global_view, saliency_view, regions, captions=None):
        # Encode images
        global_feat = self.encode_view(global_view)
        saliency_feat = self.encode_view(saliency_view)
        region_feats = [self.encode_view(region) for region in regions]

        # Concatenate features
        features = torch.cat([
            global_feat,
            saliency_feat,
            *region_feats
        ], dim=1)
        fused_feat = self.feature_fusion(features)

        # Initialize decoder
        batch_size = global_view.size(0)
        hidden = (
            torch.zeros(1, batch_size, self.decoder.hidden_size).to(device),
            torch.zeros(1, batch_size, self.decoder.hidden_size).to(device)
        )

        # Teacher forcing or inference
        seq_length = 100 if captions is None else captions.size(1)
        outputs = torch.zeros(batch_size, seq_length, self.fc.out_features).to(device)

        # Start token
        input_token = torch.zeros(batch_size, 1).long().to(device)  # <SOS>

        for t in range(seq_length):
            # Embed input token
            embedded = self.embedding(input_token)

            # Combine with image features
            combined = torch.cat([embedded, fused_feat.unsqueeze(1)], dim=2)

            # LSTM step
            output, hidden = self.decoder(combined, hidden)
            output = self.fc(output.squeeze(1))
            outputs[:, t] = output

            # Next input (teacher forcing or prediction)
            if captions is not None and torch.rand(1).item() > 0.5:
                input_token = captions[:, t].unsqueeze(1)
            else:
                input_token = output.argmax(1).unsqueeze(1)

        return outputs

    def generate(self, global_view, saliency_view, regions, max_length=100):
        self.eval()
        with torch.no_grad():
            # Encode images
            global_feat = self.encode_view(global_view)
            saliency_feat = self.encode_view(saliency_view)
            region_feats = [self.encode_view(region) for region in regions]

            # Concatenate features
            features = torch.cat([
                global_feat,
                saliency_feat,
                *region_feats
            ], dim=1)
            fused_feat = self.feature_fusion(features)

            # Initialize decoder
            batch_size = global_view.size(0)
            hidden = (
                torch.zeros(1, batch_size, self.decoder.hidden_size).to(device),
                torch.zeros(1, batch_size, self.decoder.hidden_size).to(device)
            )

            # Start token
            input_token = torch.full((batch_size, 1), vocab['<SOS>'], dtype=torch.long).to(device)
            generated = torch.zeros(batch_size, max_length, dtype=torch.long).to(device)
            finished = torch.zeros(batch_size, dtype=torch.bool).to(device)

            for t in range(max_length):
                embedded = self.embedding(input_token)
                combined = torch.cat([embedded, fused_feat.unsqueeze(1)], dim=2)
                output, hidden = self.decoder(combined, hidden)
                output = self.fc(output.squeeze(1))

                # Get predicted token
                token = output.argmax(1)
                generated[:, t] = token

                # Update finished sequences
                finished = finished | (token == vocab['<EOS>'])

                # Stop if all sequences are finished
                if finished.all():
                    break

                # Prepare next input
                input_token = token.unsqueeze(1)
                input_token[finished] = vocab['<EOS>']

            return generated



report_model = MultimodalReportGenerator(report_vocab_size=VOCAB_SIZE,hidden_dim=256 ).to(device)

# Optimizer and loss
report_optimizer = optim.Adam(report_model.parameters(), lr=1e-4)
report_criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding

# Training function for report generator
def train_report_generator(model, train_data, val_data, epochs=10):
    model.train()
    for epoch in range(epochs):
        train_loss = 0.0
        num_batches = 0

        # Shuffle training data each epoch
        import random
        random.shuffle(train_data)

        # Process in chunks to reduce memory
        for i in range(0, len(train_data), 4):  # Reduced batch size from 8 to 4
            if i + 4 > len(train_data):
                continue

            batch = train_data[i:i+4]

            # Prepare inputs
            global_views = torch.stack([item['global_view'] for item in batch]).to(device)
            saliency_views = torch.stack([item['saliency_view'] for item in batch]).to(device)
            regions = [
                torch.stack([item[f'region{i+1}'] for item in batch]).to(device)
                for i in range(3)
            ]

            # Tokenize reports
            captions = torch.stack([encode_caption(item['full_report'], vocab) for item in batch]).to(device)

            # Forward pass
            outputs = model(global_views, saliency_views, regions, captions)

            # Calculate loss
            loss = report_criterion(
                outputs.view(-1, outputs.size(-1)),
                captions.view(-1)
            )

            # Backward pass
            report_optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            report_optimizer.step()

            train_loss += loss.item()
            num_batches += 1

        avg_loss = train_loss / num_batches if num_batches > 0 else 0
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_loss:.4f}")

# Train the model with reduced epochs
train_report_generator(report_model, train_multiview, val_multiview, epochs=10)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 | Train Loss: 5.1812
Epoch 2/10 | Train Loss: 4.2544
Epoch 3/10 | Train Loss: 3.9288
Epoch 4/10 | Train Loss: 3.7367
Epoch 5/10 | Train Loss: 3.6101
Epoch 6/10 | Train Loss: 3.4972
Epoch 7/10 | Train Loss: 3.4060
Epoch 8/10 | Train Loss: 3.3267
Epoch 9/10 | Train Loss: 3.2583
Epoch 10/10 | Train Loss: 3.2274


In [12]:
################################
### 6. Evaluation & Metrics ###
################################

def calculate_bleu(generated, references):
    """Calculate BLEU scores"""
    # Tokenize reports
    refs = [[word_tokenize(ref.lower())] for ref in references]
    gens = [word_tokenize(gen.lower()) for gen in generated]

    # Calculate BLEU scores
    smoothing = SmoothingFunction().method4
    bleu1 = corpus_bleu(refs, gens, weights=(1, 0, 0, 0), smoothing_function=smoothing)
    bleu2 = corpus_bleu(refs, gens, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
    bleu3 = corpus_bleu(refs, gens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing)
    bleu4 = corpus_bleu(refs, gens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)

    return {
        "BLEU-1": bleu1,
        "BLEU-2": bleu2,
        "BLEU-3": bleu3,
        "BLEU-4": bleu4
    }

def evaluate_model(model, test_data):
    model.eval()
    generated_reports = []
    reference_reports = [item['full_report'] for item in test_data]

    # Evaluate in smaller batches to save memory
    for i in range(0, len(test_data), 4):  # Reduced batch size
        if i + 4 > len(test_data):
            continue

        batch = test_data[i:i+4]

        # Prepare inputs
        global_views = torch.stack([item['global_view'] for item in batch]).to(device)
        saliency_views = torch.stack([item['saliency_view'] for item in batch]).to(device)
        regions = [
            torch.stack([item[f'region{i+1}'] for item in batch]).to(device)
            for i in range(3)
        ]

        # Generate reports
        with torch.no_grad():
            outputs = model.generate(global_views, saliency_views, regions)

        # Decode reports (simplified)
        for output in outputs:
            words = []
            for token in output:
                token = token.item()
                if token == vocab['<EOS>']:
                    break
                if token > vocab['<UNK>']:  # Avoid special tokens
                    words.append(inv_vocab.get(token, '<UNK>'))
            generated_reports.append(" ".join(words))

    # Calculate metrics
    bleu_scores = calculate_bleu(generated_reports, reference_reports)

    # Print sample reports
    print("\nSample Report:")
    print(f"Generated: {generated_reports[0]}")
    print(f"Reference: {reference_reports[0]}")

    return bleu_scores

# Evaluate model
test_results = evaluate_model(report_model, test_multiview)
print("\nTest Results:")
for metric, score in test_results.items():
    print(f"{metric}: {score:.4f}")

# Save model
torch.save(report_model.state_dict(), "radiology_report_generator.pth")


Sample Report:
Generated: findings : the heart is normal . size . the lungs are clear . no acute cardiopulmonary abnormality .
Reference: FINDINGS:
Lungs are clear. Heart size normal. The XXXX are unremarkable.

IMPRESSION:
No acute cardiopulmonary finding.

Test Results:
BLEU-1: 0.3775
BLEU-2: 0.2746
BLEU-3: 0.2020
BLEU-4: 0.1489
